# One-dimensional packed-bed, catalytic-membrane reactor model without streamwise diffusion

The present model simulates heterogeneous catalytic processes inside packed-bed, catalytic membrane reactors. The gas-phase and surface-phase species conservation equations are derived and the system of differential-algebraic equations (DAE) is solved using the scikits.odes.dae IDA solver.

In [1]:
# Import Cantera and scikits
import numpy as np
from scikits.odes import dae
import cantera as ct
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_formats = ["svg"]
print("Runnning Cantera version: " + ct.__version__)

Runnning Cantera version: 2.6.0


### Define gas-phase and surface-phase species

In [2]:
# Import the reaction mechanism 
# mech = 'newer_hcof_c2_CFG.yaml'
mech = 'nist_pfas.yaml'
mech_surf = 'methane_Bjarne.yaml'

# Import the model for gas-phase
gas = ct.Solution(mech_surf, 'gas')

# Import the model for surf-phase
surf = ct.Interface(mech_surf,'surface1', [gas])

# Other parameters
n_gas = gas.n_species  # number of gas species
n_surf = surf.n_species  # number of surface species
n_gas_reactions = gas.n_reactions  # number of gas-phase reactions

# Set offsets of dependent variables in the solution vector
offset_rhou = 0
offset_p = 1
offset_T = 2
offset_Y = 3
offset_Z = offset_Y + n_gas
n_var = offset_Z + n_surf  # total number of variables (rhou, P, T, Yk and Zk)

print("Number of gas-phase species = ", n_gas)
print("Number of surface-phase species = ", n_surf)
print("Number of variables = ", n_var)

Number of gas-phase species =  13
Number of surface-phase species =  33
Number of variables =  49


### Define reactor geometry and operating conditions

In [3]:
# Reactor geometry
L = 5e-2  # length of the reactor (m)
R = 5e-3  # radius of the reactor channel (m)
dp = 3.37e-4  # particle diameter (m)
As = 3.5e1  # specific surface area (1/m)

# Energy (adiabatic or isothermal)
solve_energy = True  # True: Adiabatic, False: isothermal

# Get required properties based on the geometry and mechanism
W_g = gas.molecular_weights  # vector of molecular weight of gas species
area2vol = 2 / R  # area to volume ratio assuming a cylindrical reactor
D_h = 2 * R  # hydraulic diameter

# Inlet operating conditions
T_in = 673  # inlet temperature [K]
p_in = 1e5  # inlet pressure [Pa]
v_in = 0.1  # inlet velocity [m/s]
T_wall = 300  # wall temperature [K]
h_coeff = 0  # heat transfer coefficient [W/m2/K]

# Set gas and surface states
gas.TPX = T_in, p_in, "O2:0.006, CH4:0.003, N2:0.991"  # inlet composition
surf.TP = T_in, p_in
Yk_0 = gas.Y
rhou0 = gas.density * v_in

# Initial surface coverages
# advancing coverages over a long period of time to get the steady state.
surf.advance_coverages(1e10)  
Zk_0 = surf.coverages

### Define residual function required for IDA solver

In [4]:
def residual(z, y, yPrime, res):
    """Solution vector for the model
    y = [rho*u, p, T, Yk, Zk]
    yPrime = [d(rho*u)dz, dpdz, dTdz, dYkdz, dZkdz]
    """
    # Get current thermodynamic state from solution vector and save it to local variables.
    rhou = y[offset_rhou]  # mass flux (density * velocity)
    Y = y[offset_Y : offset_Y + n_gas]  # vector of mass fractions
    Z = y[offset_Z : offset_Z + n_surf] # vector of site fractions 
    p = y[offset_p]  # pressure
    T = y[offset_T]  # temperature

    # Get derivatives of dependent variables
    drhoudz = yPrime[offset_rhou]  
    dYdz = yPrime[offset_Y : offset_Y + n_gas] 
    dZdz = yPrime[offset_Z : offset_Z + n_surf] 
    dpdz = yPrime[offset_p]  
    dTdz = yPrime[offset_T] 

    # Set current thermodynamic state for the gas and surface phases
    # Note: use unnormalized mass fractions and site fractions to avoid over-constraining the system
    gas.set_unnormalized_mass_fractions(Y)
    gas.TP = T, p
    surf.set_unnormalized_coverages(Z)
    surf.TP = T, p

    # Calculate required variables based on the current state
    coverages = surf.coverages  # surface site coverages
    # heterogeneous production rate of gas species
    sdot_g = surf.get_net_production_rates("gas")
    #print(np.sum(sdot_g * W_g))
    # heterogeneous production rate of surface species
    sdot_s = surf.get_net_production_rates("surface1")
    wdot_g = np.zeros(n_gas)
    # specific heat of the mixture
    cp = gas.cp_mass  
    # partial enthalpies of gas species
    hk_g = gas.partial_molar_enthalpies  

    if n_gas_reactions > 0:
        # homogeneous production rate of gas species
        wdot_g = gas.net_production_rates  
    mu = gas.viscosity  # viscosity of the gas-phase

    # Calculate density using equation of state
    rho = gas.density

    # Conservation of total-mass
    # temporary variable
    sum_continuity = As * np.sum(sdot_g * W_g) + np.sum(wdot_g * W_g)  
    res[offset_rhou] = (drhoudz - sum_continuity)

    # Conservation of gas-phase species        
    res[offset_Y:offset_Y+ n_gas] = (dYdz + (Y * sum_continuity - np.multiply(wdot_g,W_g) 
                                     - As * np.multiply(sdot_g,W_g)) / rhou)
    
    # Conservation of site fractions (algebraic contraints in this example)
    res[offset_Z : offset_Z + n_surf] = sdot_s

    # For the species with largest site coverage (k_large), solve the constraint equation: sum(Zk) = 1
    # The residual function for 'k_large' would be 'res[k_large] = 1 - sum(Zk)'
    # Note that here sum(Zk) will be the sum of coverages for all surface species, including the 'k_large' species.
    ind_large = np.argmax(coverages)
    res[offset_Z + ind_large] = 1 - np.sum(coverages)

    # Conservation of momentum
    u = rhou / rho
    res[offset_p] = dpdz - 0 #2*rho*u*dudz + np.power(u,2)*drhodz + dpdz + 32*u*mu/D**2 

    # Conservation of energy
    res[offset_T] = dTdz - 0  # isothermal condition
    # Note: One can just not solve the energy equation by keeping temperature constant. 
    # But since 'T' is used as the dependent variable, the residual is res[T] = dTdz - 0 
    # One can also write res[T] = 0 directly, but that leads to a solver failure due to singular jacobian

    if solve_energy:
        conv_term = (4 / D_h) * h_coeff * (T_wall - T) * (2 * np.pi * R)
        chem_term = np.sum(hk_g * (wdot_g + As * sdot_g))
        res[offset_T] -= (conv_term - chem_term) / (rhou * cp)

### Calculate the spatial derivatives at the inlet that will be used as the initial conditions for the IDA solver


In [5]:
# Initialize yPrime to 0 and call residual to get initial derivatives
y0 = np.hstack((rhou0, p_in, T_in, Yk_0, Zk_0))
yprime0 = np.zeros(n_var)
res = np.zeros(n_var)
residual(0, y0, yprime0, res)
yprime0 = -res
print(res)

[ 1.00024202e-19  0.00000000e+00 -1.45482167e-13  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.98164246e-18  8.03991245e-18
 -2.05826821e-17 -1.92456409e-19 -2.41937623e-27 -9.43078479e-19
  1.56599470e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9.11815288e-26 -3.16963116e-33 -6.66133815e-16 -1.52305131e-28
  6.42040992e-47  5.20534958e-36  1.34412249e-26 -5.81333600e-35
 -1.48537718e-28  8.19459959e-58  7.01200450e-42  0.00000000e+00
  1.06065172e-34 -4.91483141e-32  1.26656831e-38 -1.30062128e-46
  5.10551157e-39  0.00000000e+00  0.00000000e+00 -1.83197155e-33
  2.34548908e-30  1.35653619e-28  6.96847747e-32  2.83680183e-21
  1.65862629e-30  9.40395435e-38  3.29244126e-28  7.76195024e-34
 -1.48540589e-28 -2.89845179e-46  1.37284801e-39  4.55723431e-46
 -1.76628679e-34]


### Solve the system of DAEs using ida solver

In [8]:
solver = dae(
    "ida",
    residual,
    first_step_size=1e-15,
    atol=1e-08,  # absolute tolerance for solution
    rtol=1e-04,  # relative tolerance for solution
    algebraic_vars_idx=[np.arange(offset_Y + n_gas, offset_Z + n_surf, 1)],
    max_steps=10000,
    one_step_compute=True,
    old_api=False,  # forces use of new api (namedtuple)
)

distance = []
solution = []
state = solver.init_step(0.0, y0, yprime0)

# Note that here the variable t is an internal varible used in scikits. In this example, it represents
# the natural variable z, which corresponds to the axial distance inside the reactor.
while state.values.t < L:
    distance.append(state.values.t)
    solution.append(state.values.y)
    state = solver.step(L)

distance = np.array(distance)
solution = np.array(solution)
#print(state)

TypeError: '<' not supported between instances of 'NoneType' and 'float'

### Plot results

In [ ]:
plt.rcParams["font.size"] = 14
f, ax = plt.subplots(3, 2, figsize=(12, 12), dpi=96)

# Plot gas pressure profile along the flow direction
ax[0, 0].plot(distance, solution[:, offset_p], color="C0")
ax[0, 0].set_xlabel("Distance (m)")
ax[0, 0].set_ylabel("Pressure (Pa)")

# Plot gas temperature profile along the flow direction
ax[0, 1].plot(distance, solution[:, offset_T], color="C1")
ax[0, 1].set_xlabel("Distance (m)")
ax[0, 1].set_ylabel("Temperature (K)")

# Plot major and minor gas species separately
minor_idx = []
major_idx = []
for j, name in enumerate(gas.species_names):
    mean = np.mean(solution[:, offset_Y + j])
    if mean <= 0.1:
        minor_idx.append(j)
    else:
        major_idx.append(j)

# Plot mass fractions of the gas-phase species along the flow direction

# Major gas-phase species
for j in major_idx:
    ax[1, 0].plot(distance, solution[:, offset_Y + j], label=gas.species_names[j])
ax[1, 0].legend(fontsize=12, loc="best")
ax[1, 0].set_xlabel("Distance (m)")
ax[1, 0].set_ylabel("Mass Fraction")

# Minor gas-phase species
for j in minor_idx:
    ax[1, 1].plot(distance, solution[:, offset_Y + j], label=gas.species_names[j])
ax[1, 1].legend(fontsize=12, loc="best")
ax[1, 1].set_xlabel("Distance (m)")
ax[1, 1].set_ylabel("Mass Fraction")

# Plot major and minor surface species separately
minor_idx = []
major_idx = []
for j, name in enumerate(surf.species_names):
    mean = np.mean(solution[:, offset_Z + j])
    if mean <= 0.1:
        minor_idx.append(j)
    else:
        major_idx.append(j)

# Plot the site fraction of the surface-phase species along the flow direction
# Major surf-phase species
for j in major_idx:
    ax[2, 0].plot(distance, solution[:, offset_Z + j], label=surf.species_names[j])
ax[2, 0].legend(fontsize=12, loc="best")
ax[2, 0].set_xlabel("Distance (m)")
ax[2, 0].set_ylabel("Site Fraction")

# Minor surf-phase species
for j in minor_idx:
    ax[2, 1].plot(distance, solution[:, offset_Z + j], label=surf.species_names[j])
ax[2, 1].legend(fontsize=12, loc="best")
ax[2, 1].set_xlabel("Distance (m)")
ax[2, 1].set_ylabel("Site Fraction")
f.tight_layout()